In [0]:
from google.colab import files
files.upload()

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('reviews.csv')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [0]:
import nltk
from nltk.tokenize import word_tokenize

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
vec = CountVectorizer(tokenizer=word_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.label)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.62      0.41      0.49       261
           0       0.31      0.18      0.22       354
           1       0.89      0.96      0.92      3134

    accuracy                           0.85      3749
   macro avg       0.61      0.52      0.55      3749
weighted avg       0.82      0.85      0.83      3749



# Веса признаков

In [0]:
clf.coef_.shape

(3, 216270)

In [0]:
print(max(enumerate(clf.coef_[0]), key=lambda pair: pair[1]),
      max(enumerate(clf.coef_[1]), key=lambda pair: pair[1]),
      max(enumerate(clf.coef_[2]), key=lambda pair: pair[1]))

(102774, 0.6401699100429972) (125744, 0.43722845722102655) (179222, 0.6142234646628411)


In [0]:
index_to_word = {
    ind: word
    for (word, ind)
    in vec.vocabulary_.items()
    }

In [0]:
print("Топ-слово для отрицательных отзывов: ", '"',
      index_to_word[102774], '"', "\n", 
      "Топ-слово для нейтральных отзывов: ", '"',
      index_to_word[125744], '"', "\n", 
      "Топ-слово для положительных отзывов: ", '"',
      index_to_word[179222], '"')

Топ-слово для отрицательных отзывов:  " никакой " 
 Топ-слово для нейтральных отзывов:  " питт " 
 Топ-слово для положительных отзывов:  " спасибо "


In [0]:
# первые 10 коэффициентов для отрицательных отзывов
top_features = sorted(enumerate(clf.coef_[0]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

никакой
скучно
отзывы
видимо
единственный
единственное
ощущение
рейтинг
вместо
нее


In [0]:
# первые 10 коэффициентов для нейтральных отзывов
top_features = sorted(enumerate(clf.coef_[1]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

питт
оставил
хорошая
как-то
увы
многое
тяжело
стороны
этих
свое


In [0]:
# первые 10 коэффициентов для положительных отзывов
top_features = sorted(enumerate(clf.coef_[2]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

спасибо
места
тарантино
задуматься
просмотру
сначала
потрясающий
рекомендую
однозначно
первый
